In [1]:
# Creiamo un regressore logistico per classificare le cifre MNIST

import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
# Abilitiamo la eager execution: cioè l'esecuzione immediata delle operazioni 
# che rende inutile il grafo di computazione e quindi la sessione
# TF 1.x non la abilita di default e quindi esegue il grafo solo alla richiesta del risultato
# La eager execution è abilitata di default in TF 2.x
tf.compat.v1.enable_eager_execution()

# log dei messaggi solo  per gli errori
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Importiamo il dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Creiamo la struttura del data set per l'addestramento del modello
def process_input_data(x_train, y_train, x_test):
    
    # stimiamo il numero delle classi e delle feature per creare un vettore monodimensionale

    num_classes = y_train.max() + 1 # le classi hanno etichette numeriche da 0 a num_classes - 1

    num_features = x_train.shape[1] * x_train.shape[2]
    
    # Convertiamo in float32.

    x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

    # Convertiamo le immagini con shape (28,28) in un vettore monodimensionale di 784 elementi (Flattening)

    x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

    # Creiamo l'embedding numerico ovvero la rappresentazione come reali in [0,1]

    x_train, x_test = x_train / 255., x_test / 255.
    
    # il vero Dataset TF è ottenuto per trasformazione di una serie di 'slice' di un tensore
    # ovvero una serie di immagini in una sequenza organizzata come una matrice 3D
    train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
    
    return (num_classes, num_features, train_data, x_train, x_test)

# Costruiamo il modello
def build_model(learning_rate,momentum,nesterov,num_features,num_classes):
    
    # definiamo il nodo di bias inizializzato a 0
    b = tf.Variable(tf.zeros([num_classes]), name="bias")

    # definiamo il nodo della matrice pesi 'W' inizializzata a 1
    W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")

    # L'ottimizzazione viene effettuata con lo Stochastic Gradient Descent
    optimizer = tf.keras.optimizers.SGD(learning_rate,momentum=momentum,nesterov=nesterov)

    return b, W, optimizer

 
# Creiamo il grafo di computazione per la regressione logistica multiclasse softmax(x*W +b)
def logistic_regression(x, W, b):
        
    # calcoliamo la regressione logistica y = softmax(x*W + b)
    # x*W + b --> logit: log-probabilità non normalizzata
    # y --> probabilità scalate in [0, 1]
    return tf.nn.softmax(tf.matmul(x, W) + b)

# Definiamo la funzione di loss 

def cross_entropy(y_pred, y_true, num_classes):

    # Codifica one-hot: un vettore binario di 10 elementi con un solo valore 1
    # per la classe predetta Es. etichetta 3 --> 0001000000

    y_true = tf.one_hot(y_true, depth=num_classes)

    # saturiamo i valori predetti per evitare il calcolo di log(0)

    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    # Calcoliamo la cross_entropia che è l'entropia "mutua" tra la distribuzione di probabilità
    # p_data rappresentata dai valori y_true e la distribuzione stimata p_model
    # rappresentata da y_pred
    #
    # H = - E_p_data[log p_model] --> 𝝨_i (- y_true_i*log y_pred_i )
    #
    # Se H è bassa allora le due distribuzioni di probabilità sono simili

    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred),axis=1))

# Definiamo la metrica di valutazione
# usiamo l'accuracy ovvero il rapporto tra le predizioni corrette e il totale
def accuracy(y_pred, y_true):

    # La classe predetta è l'indice dello score più alto nel vettore di predizione
    # l'output della softmax è una 'log-probabilità' ed è quindi un numero reale

    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))

    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# Definiamo la procedura di aggiornamento dei gradienti

def run_optimization(x, y, W, b, optimizer, num_classes):

    # Inseriamo tutta la computazione in un context manager creato da tf.GradientTape
    # per il calcolo automatico dei gradienti

    with tf.GradientTape() as g:

        pred = logistic_regression(x, W, b)

        loss = cross_entropy(pred, y, num_classes)

    # Calcoliamo i gradienti e li applichiamo
    # all'ottimizzatore 
    gradients = g.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))


#
# Procedura di addestramento principale
# 

def train_logisitc_regression(\
    x_train, y_train, x_test, y_test,\
    learning_rate = 0.01,\
    momentum=0.9,\
    nesterov=True,\
    epochs = 100,\
    batch_size = 64):
    
    
    # Preprocessing e creazione del training set
    num_classes, num_features, train_data, x_train, x_test = process_input_data(x_train, y_train, x_test)
    
        # il dataset viene ripetuto indefinitamente e impostiamo lo shuffle 
    # la diensione del batch e il prefetch dei campioni durante l'addestramento
    train_data=train_data.repeat().\
                shuffle(50000).\
                batch(batch_size).\
                prefetch(tf.compat.v1.data.experimental.AUTOTUNE) # il carico è bilanciato dinamicamente

    # Istanza del modello
    b, W, optimizer = build_model(learning_rate,momentum,nesterov,num_features,num_classes)
    
    # Eseguiamo il training
    
    num_batches = x_train.shape[0]//batch_size +1   # arrotondiamo per eccesso il numero dei batch
                                                    # pari alla dimensione del dataset / bach_size
    
    for epoch in range(1,epochs+1):
        
        loss, acc = 0.0, 0.0
        
        # in ogni epoca calcoliamo preleviamo num_batches batch
        for (batch_x, batch_y) in train_data.take(num_batches):

            # Passo di ottimizzazione
            run_optimization(batch_x, batch_y, W, b, optimizer, num_classes)

            # Metriche di training
            pred = logistic_regression(batch_x, W, b)

            loss += cross_entropy(pred, batch_y, num_classes)

            acc += accuracy(pred, batch_y)
        
        # calcoliamo i valori medi di loss e accuracy nell'epoca sui singoli batch
        loss /= num_batches
        acc /= num_batches
        
        # Metriche di test
        pred = logistic_regression(x_test, W, b) # qui W e b sono il modello addestrato
        
        test_loss = cross_entropy(pred, y_test, num_classes)
        
        test_acc = accuracy(pred,y_test)

        print(f'Epoch: {epoch:4d}, train loss: {loss:05.3f}, train accuracy: {acc:05.3f}, test loss: {test_loss:05.3f}, test accuracy: {test_acc:05.3f}')



In [2]:
train_logisitc_regression(x_train,y_train,x_test,y_test,batch_size=128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch:    1, train loss: 0.575, train accuracy: 0.862, test loss: 0.377, test accuracy: 0.900
Epoch:    2, train loss: 0.372, train accuracy: 0.898, test loss: 0.334, test accuracy: 0.909
Epoch:    3, train loss: 0.341, train accuracy: 0.905, test loss: 0.316, test accuracy: 0.914
Epoch:    4, train loss: 0.325, train accuracy: 0.909, test loss: 0.305, test accuracy: 0.916
Epoch:    5, train loss: 0.314, train accuracy: 0.912, test loss: 0.298, test accuracy: 0.918
Epoch:    6, train loss: 0.306, train accuracy: 0.914, test loss: 0.293, test accuracy: 0.919
Epoch:    7, train loss: 0.300, train accuracy: 0.915, test loss: 0.289, test accuracy: 0.920
Epoch:    8, train loss: 0.296, train accuracy: 0.917, test loss: 0.286, test accuracy: 0.921
Epoch:    9, train loss: 0.292, train accuracy: 0.918, test loss: 0.284, test accuracy: 0.921
Epoch:   10, train loss: 0.288, train accuracy: 0.919, test 

KeyboardInterrupt: 